# Entertainment Category: App

# Setting Up Necessary Things

In [1]:
# Jupyter Notebook Magic Command - Auto Reloading
%reload_ext autoreload
%autoreload 2

# Jupyter Notebook Magic Command - Inline Plotting
%matplotlib inline

In [2]:
# Ignore All Warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/All Entertainment Classification

/content/drive/MyDrive/All Entertainment Classification


# Necessary Imports

In [5]:
! pip install -q transformers[sentencepiece] fastai onnxruntime gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 99.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 14.9 MB/s eta 0:00:00

In [6]:
import numpy as np

import torch

from transformers import AutoTokenizer

import onnxruntime as rt

import gradio as gr

# Modeling

In [7]:
model_path = "models/entertainment-category-quantized.onnx"

In [8]:
categories = ['anime', 'book', 'game', 'movie', 'music', 'tv show']

In [9]:
inf_session = rt.InferenceSession('models/entertainment-category-quantized.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [10]:
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

In [60]:
def entertainment_category(description):
  input_ids = tokenizer(description)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]

  mask = np.where(probs[0] == probs.max())[0][0]
  cat = categories[mask]
  cat_prob = torch.sigmoid(torch.FloatTensor(probs))[0]

  return dict(zip(categories, map(float, cat_prob)))

In [61]:
# Anime: One Piece
desc = "Gol D. Roger was known as the 'Pirate King', the strongest and most infamous being to have sailed the Grand Line. The capture and execution of Roger by the World Government brought a change throughout the world. His last words before his death revealed the existence of the greatest treasure in the world, One Piece. It was this revelation that brought about the Grand Age of Pirates, men who dreamed of finding One Piece—which promises an unlimited amount of riches and fame—and quite possibly the pinnacle of glory and the title of the Pirate King."

In [62]:
entertainment_category(desc)

{'anime': 0.9905509948730469,
 'book': 0.6193150877952576,
 'game': 0.17966431379318237,
 'movie': 0.04237794876098633,
 'music': 0.06693349033594131,
 'tv show': 0.8937151432037354}

# Gradio Application

In [63]:
example = [
    ["March Of Soldiers is a real time strategy single player , It is a military game based on the player's skill and the strength of his financial economy"],
    ["Don Vito Corleone, head of a mafia family, decides to hand over his empire to his youngest son Michael. However, his decision unintentionally puts the lives of his loved ones in grave danger."],
    ["When I wake up, I look into the mirror, I can see a clearer, vision, I should start living today, Cause today is gonna be the day, is gonna be the day, Cause today is gonna be the day, is gonna be the day"]
]

In [67]:
label = gr.outputs.Label(num_top_classes=5)
iface = gr.Interface(fn=entertainment_category, inputs="text", outputs=label, examples=example)
iface.launch(inline=False, share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://fb30f688f6b6a30ed6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
